In [1]:
# Importing Libraries for Data Analysis and Visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import wordcloud
import nltk
import re
import string
import spacy

sns.set_theme()
%matplotlib inline

import warnings as w
w.filterwarnings('ignore')

In [2]:
airline_df = pd.read_csv('airline_df_cleaned.csv')
airline_df.head()

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended,Overall_Rating
0,air-china,China,LAX-PEK return. Food low quality staff appeare...,2012-01-30,Solo Leisure,Economy Class,Beijing to Xi'an,3,2,2,1,1,3,0,2.0
1,air-china,China,Round-trip from Hong Kong to Munich. The main ...,2012-01-31,Solo Leisure,Economy Class,Beijing to Xi'an,1,3,1,1,1,3,0,2.0
2,air-china,China,Sydney-Beijing-Paris then Rome-Beijing to Sydn...,2012-02-03,Solo Leisure,Economy Class,Beijing to Xi'an,2,3,1,2,1,3,0,2.0
3,air-china,China,London to Sydney return via Beijing. A cheap f...,2012-02-22,Solo Leisure,Economy Class,Beijing to Xi'an,4,1,2,4,3,4,0,3.0
4,air-china,China,Beijing to Shanghai. Only one check-in desk fo...,2012-02-28,Solo Leisure,Economy Class,Beijing to Xi'an,4,4,3,3,3,4,1,4.0


In [3]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15206 entries, 0 to 15205
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline                 15206 non-null  object 
 1   Country                 15206 non-null  object 
 2   Review                  15206 non-null  object 
 3   Date_Published          15206 non-null  object 
 4   Type of Traveller       15206 non-null  object 
 5   Seat Type               15206 non-null  object 
 6   Route                   15206 non-null  object 
 7   Seat Comfort            15206 non-null  int64  
 8   Cabin Staff Service     15206 non-null  int64  
 9   Food & Beverages        15206 non-null  int64  
 10  Inflight Entertainment  15206 non-null  int64  
 11  Ground Service          15206 non-null  int64  
 12  Value for Money         15206 non-null  int64  
 13  Recommended             15206 non-null  int64  
 14  Overall_Rating          15206 non-null

In [4]:
airline_df.describe(include='all')

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended,Overall_Rating
count,15206,15206,15206,15206,15206,15206,15206,15206.000000,15206.000000,15206.000000,15206.000000,15206.000000,15206.000000,15206.000000,15206.000000
unique,19,9,15205,3718,4,4,6716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,emirates,India,✅ Trip Verified | Very attentive staff at air...,2015-01-14,Solo Leisure,Economy Class,Delhi to Mumbai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2263,3637,2,39,6219,11369,739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.371169,3.502433,3.148691,3.227542,3.074905,3.365645,0.585756,3.272327
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.376056,1.547864,1.475143,1.389885,1.549484,1.527238,0.492607,1.306984
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,0.000000,2.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,1.000000,4.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,4.000000,4.000000,4.000000,5.000000,1.000000,4.000000


## Questions for Data Analysis on Airline Reviews, Ratings and Sentiment (Dashboards)
> #### 1. What is the distribution of Seat_Comfort, Cabin_Service, Food_Bev, Entertainment, Ground_Service, and Value_for_Money of each Airline?
> #### 2. What is the distribution of Type of Traveller, Seat Type, Route, and Date Flown of each airline?
> #### 3. What is the distribution of Overall Rating of each airline?
> #### 4. What is the distribution of Sentiment of each airline?
> #### 5. What is the distribution of Overall Rating of each airline by Type of Traveller, Seat Type, Country, Airline ?

In [ ]:
fig = px.histogram(airline_df, x="Sentiment", color="Sentiment", title="Sentiment Distribution",labels={'Sentiment':'Sentiment','count':'Count'},barmode='group',width=800)
fig.show()

In [ ]:
# Sentiment of each airline
fig = px.histogram(airline_df, x="Airline", color="Sentiment",
                title="Airline Wise Sentiment Distribution",
                barmode='group', histfunc='count', height=650)
fig_widget = go.FigureWidget(fig)
fig_widget
fig.show()

In [ ]:
# Pie chart of sentiment distribution
for col in ["Seat Comfort",
        "Cabin Staff Service",
        "Food & Beverages",
        "Inflight Entertainment",
        "Ground Service",
        "Value for Money",
        "Sentiment",
    ]:
        plt.pie(airline_df[col].value_counts(), labels=airline_df[col].value_counts().index,autopct='%1.1f%%', shadow=True, startangle=90)
        plt.title(col)
        plt.axis('equal')
        plt.legend()
        plt.show()

In [ ]:
# Overall Rating of each airline
fig = px.histogram(airline_df, x="Airline", color="Overall_Rating", title="Airline Wise Overall Rating Distribution",barmode='group', histfunc='count', height=650) 
fig_widget = go.FigureWidget(fig)
fig_widget
fig.show()

In [ ]:
airline_df